In [3]:
import data
import interpreter

In [4]:
arrays = data.RecordArray({
    "electrons": data.ListArray([0, 2, 4, 7], [2, 4, 7, 10], data.RecordArray({
        "pt": data.PrimitiveArray([9.21, 12.4, 19.89, 34.23, 44.13, 52.256, 82.123, 83.86, 90.98, 91.22]),
        "phi": data.PrimitiveArray([-0.24, 0.76, 1.23, 1.87, 2.24, 1.22, -0.19, -0.128, -0.94, -2.13]),
        "eta": data.PrimitiveArray([1.53404404, 1.70050254, 2.23567454, 0.71262606, 2.32647449, 0.12684006, 0.3242541 , 0.79075932, 1.87788224, 1.15080437]),
        "charge": data.PrimitiveArray([1, -1, 1, -1, 1, -1, 1, 1, -1, 1]),
        "mass": data.PrimitiveArray([0.00067409, 0.0088488 , 0.00966777, 0.00694518, 0.00624654, 0.00996599, 0.00032927, 0.00979511, 0.00908046, 0.0003503])
    })),
    "muons": data.ListArray([0, 5, 6, 7], [5, 6, 7, 10], data.RecordArray({
        "pt": data.PrimitiveArray([2.23, 4.33, 8.56, 9.11, 11.21, 12.23, 19.46, 34.67, 42.92, 44.45]),
        "phi": data.PrimitiveArray([-2.34, -1.16, 1.29, 0.27, -0.11, -0.56, -1.17, -1.56, 1.28, 1.44]),
        "eta": data.PrimitiveArray([0.69633868, 1.83683534, 0.12403148, 2.1757794 , 1.01284572, 0.08542841, 0.43083165, 2.38795639, 1.02827405, 1.73510134]),
        "charge": data.PrimitiveArray([-1, -1, -1, -1, 1, 1, 1, 1, -1, 1]),
        "mass": data.PrimitiveArray([0.105712890625,0.105712890625,0.105712890625,0.105712890625,0.105712890625,0.105712890625,0.105712890625,0.105712890625,0.105712890625,0.105712890625])
    })),
    "jets": data.ListArray([0, 4, 9, 17], [4, 9, 17, 21], data.RecordArray({
        "pt": data.PrimitiveArray([91.10162809, 51.80506389, 16.73566404, 76.0951384 , 73.80264364,
                                    42.17740146, 70.68699547, 68.26175309, 95.99928512, 51.77467093,
                                    28.94306475, 21.18425974, 16.32143945, 92.27924512, 95.75173021,
                                    12.06435678, 78.90351872, 83.28129708, 51.49542951, 45.2625673 ,
                                    58.16771839]),
        "eta": data.PrimitiveArray([1.32386677, 1.83170278, 0.97159418, 1.00780743, 2.27206966,
                                    2.04664609, 0.48551661, 0.37916271, 0.85091026, 0.82166978,
                                    0.29009542, 2.13980567, 1.52554065, 0.72676887, 0.30823791,
                                    1.47296837, 0.40937381, 1.95231267, 2.01706775, 1.30984769,
                                    0.84316705])
    }))
})

arrays.setindex()
instances = data.instantiate(arrays)

In this analysis, events were divided into four channels: mumu, ee, emu, and three-lepton. While the two-lepton channels were relatively easy to resolve with Coffea, some issues arose in the three-lepton channel. Going to event loop-style programming in Python/numba was a solution; PartiQL is a lot easier.

The first of these was getting the lowest-pT lepton of each event. If they're same-flavor, this would be relatively easy, since they're sorted by pT, but when they start to mix, things get trickier. My old solution took as input a mix of leptons, ran through them per-event, and then put either a 1 in an e_mask or mu_mask, which was then applied to the general e's and mu's and the lowest was taken (since the respective lepton arrays are pT-sorted).

In [ ]:
def channel_subpt(e_starts, e_stops, mu_starts, mu_stops,
                  e_pt, mu_pt):
    e_mask = np.zeros(e_starts.size, dtype=np.bool_)
    mu_mask = np.zeros(mu_starts.size, dtype=np.bool_)
    for e_start, e_stop, mu_start, mu_stop, i in zip(e_starts, e_stops, mu_starts, mu_stops, range(0, e_starts.size)):
        event_pt = np.concatenate((e_pt[e_start:e_stop], mu_pt[mu_start:mu_stop]))
        if len(event_pt) > 0:
            index = event_pt.argmin()
            if index in range(0, e_stop-e_start):
                e_mask[i] = 1
            else:
                mu_mask[i] = 1
    return e_mask, mu_mask

Now, compared to the PartiQL solution, it's bulky, ugly, and takes a lot more time to figure out:

In [8]:
minpt, counter = interpreter.run(r"""

cut count(electrons union muons) == 3 {
    third =  (electrons union muons) min by pt
}

""", instances)

minpt

List{ 
    Rec#4(1){ 
        third = Rec#6(1 0){ 
            pt = Value#6(1 0){ 12.23 }
            phi = Value#6(1 0){ -0.56 }
            eta = Value#6(1 0){ 0.08542841 }
            charge = Value#6(1 0){ 1 }
            mass = Value#6(1 0){ 0.105712890625 }
        }
    }
}

Another complication occurred in trying to plot dphi of the highest pT SSLP. This one felt like a bigger problem, as the solution I had didn't output a mask like most of the others, but rather selected the pair in the loop and outputted an array of the actual dphi's. Of course, consistency would be nice.

In [ ]:
def channel_dphi_ssp(e_starts, e_stops, mu_starts, mu_stops,
                     e_px, e_py, e_phi, mu_px, mu_py, mu_phi, 
                     e_charge, mu_charge):
    dphi = np.zeros(e_starts.size)
    dphi.fill(-np.inf)
    for e_start, e_stop, mu_start, mu_stop, i in zip(e_starts, e_stops, mu_starts, mu_stops, range(0, e_starts.size)):
        event_px = np.concatenate((e_px[e_start:e_stop], mu_px[mu_start:mu_stop]))
        event_py = np.concatenate((e_py[e_start:e_stop], mu_py[mu_start:mu_stop]))
        event_phi = np.concatenate((e_phi[e_start:e_stop], mu_phi[mu_start:mu_stop]))
        event_charge = np.concatenate((e_charge[e_start:e_stop], mu_charge[mu_start:mu_stop]))
        maxpt = -np.inf
        pair = (None, None)
        for combination in np.array(combinations(np.argsort(event_px))):
            if (np.abs(event_charge[combination[0]] + event_charge[combination[1]]) == 2):
                pair_pt = ptAddition(event_px[combination[0]], event_py[combination[0]], event_px[combination[1]], event_px[combination[1]])
                if pair_pt > maxpt:
                    maxpt = pair_pt
                    pair = (combination[0], combination[1])
        if pair != (None, None):
            dphi[i] = np.abs(event_phi[pair[0]] - event_phi[pair[1]])
    return dphi[np.invert(np.isinf(dphi))]

Regardless, you can easily see that this selection is much nicer in PartiQL syntax.

In [10]:
dphi, counter = interpreter.run(r"""

cut count(electrons union muons) == 3 {
    sslp = electrons as (lep1, lep2) union muons as (lep1, lep2) where lep1.charge == lep2.charge max by (lep1.pt + lep2.pt)
    dphi = abs(sslp.lep1.phi - sslp.lep2.phi)
    
    hist dphi
}

""", instances)

dphi

List{ 
    Rec#4(1){ 
        sslp = Rec#5*#5(1 0){ 
            lep1 = Rec#5(1 0){ 
                pt = Value#5(1 0){ 19.89 }
                phi = Value#5(1 0){ 1.23 }
                eta = Value#5(1 0){ 2.23567454 }
                charge = Value#5(1 0){ 1 }
            }
            lep2 = Rec#5(1 1){ 
                pt = Value#5(1 1){ 34.23 }
                phi = Value#5(1 1){ 1.87 }
                eta = Value#5(1 1){ 0.71262606 }
                charge = Value#5(1 1){ -1 }
            }
        }
        dphi = Value#4(1){ 0.6400000000000001 }
    }
}

A similar problem to the one with dphi arose with deta between the max forward jet and the closest lepton. I selected the forward jets in Coffea and had them as an input to this function, then selected them by the index of the event (which was a parameter in the loop). 

In [ ]:
def channel_fwdeta_jetlep(e_starts, e_stops, mu_starts, mu_stops,
                          jet_maxeta, e_eta, mu_eta):
    deta = np.zeros(e_starts.size)
    deta.fill(-np.inf)
    for e_start, e_stop, mu_start, mu_stop, i in zip(e_starts, e_stops, mu_starts, mu_stops, range(0, e_starts.size)):
        jet = jet_maxeta[i]
        event_eta = np.concatenate((e_eta[e_start:e_stop], mu_eta[mu_start:mu_stop]))
        deta[i] = np.min(np.abs(jet - event_eta))
    return deta

The Python code was surprisingly short in this case, but not as clear or nice as the PartiQL code below. It also eliminated thinking too hard about what input is needed, since we can get stuff like the max forward jet as a query.

In [11]:
deta, counter = interpreter.run(r"""

cut count(electrons union muons) == 3 {
    leptons = electrons union muons
    fwdjet = jets max by eta
    closest = leptons min by abs(fwdjet.eta - eta)
    deta = abs(closest.eta - fwdjet.eta)
    
    hist deta
}

""", instances)

deta

List{ 
    Rec#4(1){ 
        leptons = List#4(1){ 
            Rec#5(1 0){ 
                pt = Value#5(1 0){ 19.89 }
                phi = Value#5(1 0){ 1.23 }
                eta = Value#5(1 0){ 2.23567454 }
                charge = Value#5(1 0){ 1 }
                mass = Value#5(1 0){ 0.00966777 }
            }
            Rec#5(1 1){ 
                pt = Value#5(1 1){ 34.23 }
                phi = Value#5(1 1){ 1.87 }
                eta = Value#5(1 1){ 0.71262606 }
                charge = Value#5(1 1){ -1 }
                mass = Value#5(1 1){ 0.00694518 }
            }
            Rec#6(1 0){ 
                pt = Value#6(1 0){ 12.23 }
                phi = Value#6(1 0){ -0.56 }
                eta = Value#6(1 0){ 0.08542841 }
                charge = Value#6(1 0){ 1 }
                mass = Value#6(1 0){ 0.105712890625 }
            }
        }
        fwdjet = Rec#7(1 0){ 
            pt = Value#7(1 0){ 73.80264364 }
            eta = Value#7(1 0){ 2.27206966 }
        }
 